In [49]:
from openai import OpenAI
from sk import my_sk
import time


In [50]:
client  = OpenAI(api_key=my_sk)

In [51]:
def wait_for_assitant(thread, run):
    t0 = time.time()
    while run.status != "completed":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(0.25)
    dt = time.time() - t0
    print(f"Run completed in {dt:.2f} seconds")
    return run

In [52]:
vector_store = client.beta.vector_stores.create(name="Rules")

file_paths = ["Laws of the Game 2023_24.pdf", "16005001.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)
print(file_batch.status)
print(file_batch.file_counts)


completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


In [53]:
instructions_string = "RulesGPT is a Football rules assistant. \
It provides information about any rules related to football in a concise and easy to understand way. \
If a question is asked about a specific rule in Football, it provides the information so that the rule can be understood completely without any further research."

assistant = client.beta.assistants.create(
    name="RulesGPT",
    description="Football rules assistant",
    instructions=instructions_string,
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
    model = "gpt-4o")

In [54]:
print(assistant)

Assistant(id='asst_m524suc4kcC0IIKsyzwvff45', created_at=1716214819, description='Football rules assistant', instructions='RulesGPT is a Football rules assistant. It provides information about any rules related to football in a concise and easy to understand way. If a question is asked about a specific rule in Football, it provides the information so that the rule can be understood completely without any further research.', metadata={}, model='gpt-4o', name='RulesGPT', object='assistant', tools=[FileSearchTool(type='file_search')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_RaIZ8Kmp17tusB5t1Z7JX56C'])), top_p=1.0)


In [55]:
thread = client.beta.threads.create()

user_message = "Can a player be offside from a goal kick?"

message = client.beta.threads.messages.create(thread_id=thread.id, role="user", content=user_message)

run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

In [56]:
run = wait_for_assitant(thread, run)

Run completed in 1.72 seconds


In [28]:
run

Run(id='run_1ZGNaLc6Q9dP3mt59tYNxI9l', assistant_id='asst_zGlUcin8Wm8QfZEpZXVYfmwZ', cancelled_at=None, completed_at=1716211980, created_at=1716211977, expires_at=None, failed_at=None, incomplete_details=None, instructions='RulesGPT is a Football rules assistant. It provides information about any rules related to football in a concise and easy to understand way. If a question is asked about a specific rule in Football, it provides the information so that the rule can be understood completely without any further research.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=1716211977, status='completed', thread_id='thread_J7XAPhQGHpQWYj3uIQdAzoki', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=Usage(completion_tokens=172, prompt_tokens=90, total_tokens=262), temperature=1.0, top_p=1.0, tool_resources=

In [57]:
message = client.beta.threads.messages.list(thread_id=thread.id)
print(message.data[0].content[0].text.value)

According to the Laws of the Game as set out by the International Football Association Board (IFAB), a player cannot be offside from a goal kick. This is explicitly stated in Law 11, which governs offside situations.

Here's the relevant section from Law 11:

>A player is not in an offside position if he receives the ball directly from a goal kick, a corner kick, or a throw-in.

So, if a player receives the ball directly from a goal kick, they are exempt from being called offside, regardless of their position on the field at the time the kick is taken. This rule applies to both offensive and defensive players.
